In [5]:
import matplotlib
matplotlib.use('Agg')

import pandas as pd
from tqdm import tqdm
import numpy as np
import random
import cv2
import matplotlib.pyplot as plt
from glob import glob

from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import jaccard_score
from pathlib import Path

import sys
sys.path.append('../')
from torchlib.metrics import get_metrics

import staple

In [6]:
def show(src, titles=[], suptitle="", 
         bwidth=4, bheight=4, save_file=False,
         show_axis=True, show_cbar=False):

    num_cols = len(src)
    
    plt.figure(figsize=(bwidth * num_cols, bheight))
    plt.suptitle(suptitle)

    for idx in range(num_cols):
        plt.subplot(1, num_cols, idx+1)
        if not show_axis: plt.axis("off")
        if idx < len(titles): plt.title(titles[idx])
        plt.imshow(src[idx]*1)
        if type(show_cbar) is bool:
            if show_cbar: plt.colorbar()
        elif idx < len(show_cbar) and show_cbar[idx]:
            plt.colorbar()
        
    plt.tight_layout()
    if save_file:
        plt.savefig(save_file)
        
def show2(src, titles=[], suptitle="", show_fig=True,
         bwidth=4, bheight=4, save_file=False,
         show_axis=True, show_cbar=False):

    num_cols = len(src)
    
    fig = plt.figure(figsize=(bwidth * num_cols, bheight*2))
    plt.suptitle(suptitle)

    add = 0
    for idx in range(num_cols):
        if idx == 6:
            add = 2
            
        plt.subplot(2, int(np.ceil(num_cols//2)+1), idx+1+add)
        if not show_axis: plt.axis("off")
        if idx < len(titles): plt.title(titles[idx])
        plt.imshow(src[idx]*1)
        if type(show_cbar) is bool:
            if show_cbar: plt.colorbar()
        elif idx < len(show_cbar) and show_cbar[idx]:
            plt.colorbar()
        
    plt.tight_layout()
    if save_file:
        Path(save_file).parent.mkdir(exist_ok=True, parents=True)
        plt.savefig(save_file)
        
    if not show_fig:
        plt.close(fig)

# Data

In [7]:
def dice_score(gt, seg):
    jac = jaccard_score(gt.ravel(), seg.ravel())
    dice = (2*jac)/ (1+jac)
    return dice

def to3(src):
    shape = src.shape
    return np.stack((np.zeros_like(src), 1-src, src), axis=2)

def get_title(src, gt, prefix="", ret_score=False):
    results, n_cells, preds = get_metrics(to3(gt), to3(src), post_label='map')
    predictionlb, prediction, region, output = preds
    title = f"Np: {results['n_cells']} PQ: {results['pq']:0.2f}"
    if ret_score:
        return prefix + title, results['pq']
    return prefix + title



In [8]:
plt.rcParams.update({'font.size': 20})

def get_nw_simple(segs):
    new_segs = np.array(segs).copy()
    results, n_segs = [], []
    alfa   = 1
    mdices = []
    titles = []

    for epoch in range(4):
        MV = (new_segs.sum(0)>= new_segs.shape[0]//2)
        titles.append(get_title(MV, gt, f"It {epoch} Ni {new_segs.shape[0]} "))
        results.append(MV)
        n_segs.append(len(new_segs))
        seg_score = []
        for seg in new_segs:
            jac = jaccard_score(MV.ravel(), seg.ravel())
            dice = (2*jac)/ (1+jac)
            seg_score.append([dice, seg])


        seg_score  = np.array(seg_score, dtype=object)
        #sorted_seg = np.array(sorted(seg_score, key=lambda x:x[0], reverse=True))
        wdice = seg_score[:, 0].sum()
        mdice = seg_score[:, 0].mean()
        sdice = seg_score[:, 0].std()

        t = mdice - (alfa*sdice)
        new_segs = np.array((list(filter(lambda x: x[0]> t, seg_score))))
        if new_segs.size ==0:break
        new_segs = new_segs[:, 1]
    return MV

def get_simple(segs, alfa=0.2):
    new_segs = np.array(segs).copy()
    results, n_segs = [], []
    
    MV = (new_segs.sum(0)>= new_segs.shape[0]//2)
    new_weights = np.ones(len(new_segs))
    mdices = []
    titles = []


    for epoch in range(6):

        MV = (new_segs.transpose(1,2,0) * new_weights).sum(2) >= (new_weights.sum()/2)
        titles.append(get_title(MV, gt, f"It {epoch} Ni {new_segs.shape[0]} "))
        results.append(MV)
        n_segs.append(len(new_segs))

        dices = []
        for seg in new_segs:
            jac = jaccard_score(MV.ravel(), seg.ravel())
            dice = (2*jac)/ (1+jac)
            dices.append(dice)

        dices  = np.array(dices)
        #sorted_seg = np.array(sorted(seg_score, key=lambda x:x[0], reverse=True))
        wdice = dices.sum()
        mdice = dices.mean()
        sdice = dices.std()
        mdices.append(mdice)

        t = mdice - (alfa * sdice)
        new_segs = new_segs[dices > t]
        if new_segs.size ==0:break
        new_weights = dices[dices > t]
        if len(new_segs)<=1:
            break
        return MV

In [ ]:
dataset_name = 'Seg1009_0.4.1'
for sub_set in ['test', 'val', 'train']:
    img_ids = [Path(url).stem for url in (sorted(glob(f"/home/chcp/Datasets/{dataset_name}/{sub_set}/outputs/000/*")))]

    n_cells, scores_mv, scores_bm = [], [], []
    scores_staple, scores_nw_simple, scores_simple = [], [], []

    for idi, img_id in enumerate(tqdm(img_ids)):
        segs = ([cv2.imread(url, -1)//255 for url in sorted(glob(f"/home/chcp/Datasets/{dataset_name}/{sub_set}/outputs/*/{img_id}.tif"))])
        n_segs = len(segs)
        gt = cv2.imread(f"/home/chcp/Datasets/{dataset_name}/{sub_set}/labels2c/{img_id}.tif", -1)
        gt = cv2.dilate(gt, np.ones((3,3)))
        mv = np.array(segs).sum(0) >= (n_segs//2)
        _, n_cell, preds = get_metrics(to3(gt), to3(gt), post_label='map')

        segs_title_score = [get_title(src, gt, f"Seg{idx} ", ret_score=True) for idx, src in enumerate(segs)]

        segs_title_score = np.array(segs_title_score)
        title_segs = segs_title_score[:, 0]
        score_bm   = segs_title_score[:, 1].astype(float).max()

        title_mv, score_mv   = get_title(mv, gt, "MV ", ret_score=True)



        n_cells.append(n_cell)
        scores_mv.append(score_mv)
        scores_bm.append(score_bm)

        staple_res = staple.STAPLE(list(segs)).run()
        staple_res = staple_res > 0.5
        title_staple, score_staple = get_title(staple_res, gt, prefix="STAPLE ",ret_score=True)
        scores_staple.append(score_staple)


        nw_simple_seg = get_nw_simple(list(segs))
        title_nw_simple, score_nw_simple = get_title(nw_simple_seg, gt, prefix="NW SIMPLE ", ret_score=True)
        scores_nw_simple.append(score_nw_simple)

        simple_seg = get_simple(list(segs))
        title_simple, score_simple = get_title(simple_seg, gt, prefix="SIMPLE ", ret_score=True)
        scores_simple.append(score_simple)


        show_list = [gt, mv, staple_res, nw_simple_seg, simple_seg] + segs 
        show_titles= [f"GT N {n_cell}", title_mv, title_staple, title_nw_simple, title_simple] + list(title_segs)


        show2(show_list, titles=show_titles, bheight=7, bwidth=4, show_axis=False, save_file=f"consensus/{dataset_name}/{sub_set}/{img_id}.jpg", show_fig=False)

    n_cells = np.array(n_cells)
    scores_mv = np.array(scores_mv)
    scores_bm = np.array(scores_bm)
    scores_staple = np.array(scores_staple)
    scores_nw_simple = np.array(scores_nw_simple)
    scores_simple = np.array(scores_simple)
    
    df = pd.DataFrame()
    df['set'] = f'{dataset_name}-{sub_set}'
    df['scores_mv'] = [(scores_mv * n_cells).sum()/np.sum(n_cells)]
    df['scores_bm'] = [(scores_bm * n_cells).sum()/np.sum(n_cells)]
    df['scores_staple'] = [(scores_staple * n_cells).sum()/np.sum(n_cells)]
    df['scores_nw_simple'] = [(scores_nw_simple * n_cells).sum()/np.sum(n_cells)]
    df['scores_simple'] = [(scores_simple * n_cells).sum()/np.sum(n_cells)]
    df.to_csv(f'consensus/{dataset_name}-{sub_set}_simple.csv')

In [ ]:
dataset_name = 'Seg33_1.0.4'
for sub_set in ['test']:
    img_ids = [Path(url).stem for url in (sorted(glob(f"/home/chcp/Datasets/{dataset_name}/{sub_set}/outputs/001/*")))]

    n_cells, scores_mv, scores_bm = [], [], []
    scores_staple, scores_nw_simple, scores_simple = [], [], []
    

    for idi, img_id in enumerate(tqdm(img_ids)):
        segs = ([cv2.imread(url, -1)//255 for url in sorted(glob(f"/home/chcp/Datasets/{dataset_name}/{sub_set}/outputs/*/{img_id}.tif"))])
        n_segs = len(segs)
        gt = cv2.imread(f"/home/chcp/Datasets/{dataset_name}/{sub_set}/labels2c/{img_id}.tif", -1)
        gt = cv2.dilate(gt, np.ones((3,3)))
        mv = np.array(segs).sum(0) >= (n_segs//2)
        _, n_cell, preds = get_metrics(to3(gt), to3(gt), post_label='map')

        segs_title_score = [get_title(src, gt, f"Seg{idx} ", ret_score=True) for idx, src in enumerate(segs)]

        segs_title_score = np.array(segs_title_score)
        title_segs = segs_title_score[:, 0]
        score_bm   = segs_title_score[:, 1].astype(float).max()

        title_mv, score_mv   = get_title(mv, gt, "MV ", ret_score=True)



        n_cells.append(n_cell)
        scores_mv.append(score_mv)
        scores_bm.append(score_bm)

        staple_res = staple.STAPLE(list(segs)).run()
        staple_res = staple_res > 0.5
        title_staple, score_staple = get_title(staple_res, gt, prefix="STAPLE ",ret_score=True)
        scores_staple.append(score_staple)


        nw_simple_seg = get_nw_simple(list(segs))
        title_nw_simple, score_nw_simple = get_title(nw_simple_seg, gt, prefix="NW SIMPLE ", ret_score=True)
        scores_nw_simple.append(score_nw_simple)

        simple_seg = get_simple(list(segs))
        title_simple, score_simple = get_title(simple_seg, gt, prefix="SIMPLE ", ret_score=True)
        scores_simple.append(score_simple)


        show_list = [gt, mv, staple_res, nw_simple_seg, simple_seg] + segs 
        show_titles= [f"GT N {n_cell}", title_mv, title_staple, title_nw_simple, title_simple] + list(title_segs)


        #show2(show_list, titles=show_titles, bheight=7, bwidth=4, show_axis=False, save_file=f"consensus/{dataset_name}/{sub_set}/{img_id}.jpg", show_fig=False)

    n_cells = np.array(n_cells)
    scores_mv = np.array(scores_mv)
    scores_bm = np.array(scores_bm)
    scores_staple = np.array(scores_staple)
    scores_nw_simple = np.array(scores_nw_simple)
    scores_simple = np.array(scores_simple)
    
    df = pd.DataFrame()
    df['set'] = f'{dataset_name}-{sub_set}'
    df['scores_mv'] = [(scores_mv * n_cells).sum()/np.sum(n_cells)]
    df['scores_bm'] = [(scores_bm * n_cells).sum()/np.sum(n_cells)]
    df['scores_staple'] = [(scores_staple * n_cells).sum()/np.sum(n_cells)]
    df['scores_nw_simple'] = [(scores_nw_simple * n_cells).sum()/np.sum(n_cells)]
    df['scores_simple'] = [(scores_simple * n_cells).sum()/np.sum(n_cells)]
    df.to_csv(f'consensus/{dataset_name}-{sub_set}_simple.csv')

In [ ]:
np.sum(n_cells)

In [57]:
df = pd.read_csv("../extra/summary.csv")
df = df[df.subset == 'train']
df.head()

,dataset,subset,model,post,WPQ,WSQ,WRQ,PCells,Cells,Images
6,FluoC2DLMSC_0.1.1,train,/home/chcp/Documents/Mestrado/MedicalImageSegm...,th,0.628784,0.811265,0.769326,754,760,109
7,FluoC2DLMSC_0.1.1,train,/home/chcp/Documents/Mestrado/MedicalImageSegm...,wts,0.620728,0.803648,0.764296,699,760,109
8,FluoC2DLMSC_0.1.1,train,/home/chcp/Documents/Mestrado/MedicalImageSegm...,map,0.590737,0.814801,0.720322,871,760,109
15,FluoC2DLMSC_0.1.1,train,/home/chcp/Documents/Mestrado/MedicalImageSegm...,th,0.622620,0.808980,0.764163,785,760,109
16,FluoC2DLMSC_0.1.1,train,/home/chcp/Documents/Mestrado/MedicalImageSegm...,wts,0.602604,0.797901,0.746192,719,760,109


In [69]:
df.shape

(402, 10)

In [68]:
df.WPQ.describe()

count    402.000000
mean       0.471007
std        0.178943
min        0.000000
25%        0.403032
50%        0.550578
75%        0.586982
max        0.662747
Name: WPQ, dtype: float64

In [76]:
df.boxplot()

<AxesSubplot:>

In [59]:
worses = df[df.WPQ < 0.2].sort_values("WPQ", ascending=False).reset_index(drop=True)

In [64]:
worses.head()

,dataset,subset,model,post,WPQ,WSQ,WRQ,PCells,Cells,Images
0,FluoC2DLMSC_0.1.1,train,/home/chcp/Documents/Mestrado/MedicalImageSegm...,wts,0.126197,0.550514,0.209490,2387,760,109
1,FluoC2DLMSC_0.1.1,train,/home/chcp/Documents/Mestrado/MedicalImageSegm...,th,0.115572,0.590414,0.187771,3348,760,109
2,FluoC2DLMSC_0.1.1,train,/home/chcp/Documents/Mestrado/MedicalImageSegm...,th,0.111435,0.447944,0.167957,1032,760,109
3,FluoC2DLMSC_0.1.1,train,/home/chcp/Documents/Mestrado/MedicalImageSegm...,wts,0.097499,0.420802,0.146455,879,760,109
4,FluoC2DLMSC_0.1.1,train,/home/chcp/Documents/Mestrado/MedicalImageSegm...,th,0.097308,0.294725,0.138937,649,760,109


In [ ]:
baseline_unetpad_wce_step_SAW_adam_FluoC2DLMSC_0.1.1-1_map_0001_101_chk000013

In [62]:
worses.model[0]

'/home/chcp/Documents/Mestrado/MedicalImageSegmentation/Projects/pytorch-unet/out/Fluo/baseline_unetpad_wce_step_SAW_adam_FluoC2DLMSC_0.1.1-1_map_0001_101/models/chk000013.pth.tar'